In [3]:
#We compute the largest Colex graph which is subgraph of B
def LargestColex(B):
    b=B.clique_number()
    C=graphs.CompleteGraph(b)
    score=binomial(b,2);
    for i in range(0,b):
        C.add_edge(b,i)
        if B.subgraph_search_count(C)>0:
            score+=1
        else:
            break
    return score

#we compute the precise outcomes for the colex-, max degree - and VC game given the graphs coloured by A and B resp.
def winning_col(A,B):
    a=LargestColex(A)
    b=LargestColex(B)
    return 11*a-10*b

def winning_maxdeg(A,B):
    a=max(A.degree_sequence())
    b=max(B.degree_sequence())
    return 11*a-10*b

#for VC, we also mention the number of the vertices in the initial graph (N) in the function
def winning_VC(A,B,N):
    a=0;
    b=0;
    for i in range(0,N):
        if A.degree(i)>B.degree(i):
            a+=1;
        elif A.degree(i)<B.degree(i):
            b+=1;
    return 11*a-10*b

#This function converts the associated score into the outcome (points of both players)
def pair(n):
    return (n%10,n%10-floor(n/10))

In [4]:
import time

#We build Colex graphs on N vertices and binomial(N-1,2)+i and play the games for these 
for N in range(2,6):
    for i in range(1,N):
        #the size is computed and printed
        m=binomial(N-1,2)+i;
        print("**"+str(m)+"**")

        #we construct a multigraph representing the Colex graph on which the game is played
        #Here the edges that are not present in the Colex graph, have multiplicity 3
        tijd1=time.time()
        G = Graph(multiedges=True)
        for k in range(0,N):
            G.add_vertex(k)
        for i in range(1,binomial(N,2)-m+1):
            G.add_edge((0,i));
            G.add_edge((0,i));
            G.add_edge((0,i));

        J=G.copy();
        #We define alle possibilities up to isomorphism for the first edge by player A (represented by single edge)
        d={}
        d[0]=[G.canonical_label().copy(immutable=True)]
        n=1
        L={}
        for (u,v) in Combinations(range(0,N),2):
            if (u,v) not in G.edges(labels=False):
                H=G.copy(immutable=False)
                H.add_edge((u,v));
                J=H.canonical_label().copy(immutable=True)
                L[J]=1
        d[1]=list(L.keys())
        
        #in the next steps, the dictionary d[i] collects all plausible intermediate colourings of the Colex graph
        #where i edges are coloured; the multiplicity (1 or 2) corresponds with the player (1 or 2 resp.)
        #by considering a canonical labeling, we can check that we only memory non-isomorphic copies
        for n in range(2,m+1):
            L={}
            for J in d[n-1]:
                for (u,v) in Combinations(range(0,N),2):
                    if (u,v) not in J.edges(labels=False):
                        H=J.copy(immutable=False)
                        if n%2==1:
                            H.add_edge((u,v));
                        if n%2==0:
                            H.add_edge((u,v));
                            H.add_edge((u,v));
                        G2=H.canonical_label().copy(immutable=True)
                        L[G2]=1
            d[n]=list(L.keys())

            
        #We compute the scores of the 3 games for every final configuration (colouring)
        #W will contain all info of intermediate games with n edges coloured. 
        #In every loop, w is a dictionary containing the scores for games with a fixed number of edges n. 
        W={}
        w={}
        for j in d[m]:
            E=j.edges(labels=False)
            temp={}
            for i in E:
                temp[i]=0
            for i in E:
                temp[i]+=1;

            A=Graph()
            B=Graph()
            for i in range(0,N):
                A.add_vertex(i)
                B.add_vertex(i)
            for i in E:
                if temp[i]==1:
                    A.add_edge(i)
                if temp[i]==2:
                    B.add_edge(i)
            w[j]=[winning_col(A,B), winning_maxdeg(A,B), winning_VC(A,B,N)];
        W[m]=w 
        
        #Iteratively, we compute for every configuration in the intermediate stages the score at the end
        #(when the two players play optimal, 1st one wants to maximize the score, and the second one to minimize it)
        for n in range(m-1,-1,-1):
            w={}
            if n%2==1:
                for j in d[n]:
                    wins=[50000,50000,50000];
                    G=j
                    for (u,v) in Combinations(range(0,N),2):
                        if (u,v) not in G.edges(labels=False):
                            H=G.copy(immutable=False)
                            H.add_edge((u,v));
                            H.add_edge((u,v));
                            J=H.canonical_label().copy(immutable=True)
                            for i in range(3):
                                wins[i]=min(W[n+1][J][i],wins[i]);
                    w[j]=wins;

            else:
                for j in d[n]:
                    wins=[-50000,-50000,-50000];
                    G=j
                    for (u,v) in Combinations(range(0,N),2):
                        if (u,v) not in G.edges(labels=False):
                            H=G.copy(immutable=False)
                            H.add_edge((u,v));
                            J=H.canonical_label().copy(immutable=True)
                            for i in range(3):
                                wins[i]=max(W[n+1][J][i],wins[i]);
                    w[j]=wins;
#             print(n)
            W[n]=w
#             print(len(W[n]))

        #
        x=list(W[0].values())[0]
        print((pair(x[0]),pair(x[1]),pair(x[2])));
        
##Time to execute can be printed as well
#         tijd2=time.time()
#         print(tijd2-tijd1)

**1**
((1, 0), (1, 0), (2, 0))
**2**
((1, 1), (1, 1), (1, 1))
**3**
((2, 1), (2, 1), (1, 0))
**4**
((2, 2), (2, 2), (2, 1))
**5**
((2, 1), (2, 1), (2, 0))
**6**
((2, 2), (2, 2), (2, 2))
**7**
((2, 2), (3, 2), (3, 2))
**8**
((2, 2), (3, 2), (2, 1))
**9**
((4, 2), (3, 2), (2, 1))
**10**
((4, 4), (3, 3), (1, 1))
